In [511]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")



In [512]:
conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)

In [513]:
transactions = pd.read_sql("SELECT * FROM transactions", db)
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [514]:
def lookupIdByName(nameString):
    nameMatch = senators[senators['senator'].str.contains(nameString)]

    if len(nameMatch)>1:
        return nameMatch
    else: 
        return nameMatch['senatorId']


lookupIdByName("Tuber")

0    100
Name: senatorId, dtype: int64

In [515]:
tub  = transactions[transactions['senatorId']==100]
tub.ticker.value_counts().head(15)


--      88
ECOM    30
CLF     28
X       26
PYPL    24
OXY     20
INTC    19
MSFT    15
SSYS    15
BABA    14
AA      14
CALX    10
QCOM     9
F        8
NU       6
Name: ticker, dtype: int64

In [516]:
tub[(tub['ticker']=='QCOM') & (tub['type']=='Purchase') 
& (tub['asset_type']=='Stock')
]


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
261,06/16/2022,Joint,QCOM,QUALCOMM Incorporated - Common Stock,Stock,Purchase,"$250,001 - $500,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/13/2022,261,100
1007,03/29/2021,Joint,QCOM,QUALCOMM Incorporated,Stock,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/23/2021,1002,100


In [517]:

def reformatDate(transDate):
    listDate  = transDate.split("/")
    return f"{listDate[2]}-{listDate[0]}-{listDate[1]}"

transactions['transaction_date'] = transactions['transaction_date'].apply(lambda row: reformatDate(row))

transactions['transaction_date'].head()


0    2022-10-28
1    2022-10-31
2    2022-10-28
3    2022-10-28
4    2022-10-24
Name: transaction_date, dtype: object

In [518]:
for frame in [historicalPricesAG, historicalPricesHR, historicalPricesSZ]:
    frame['date'] = pd.to_datetime(frame['date'])

qc = historicalPricesHR[['QCOM', 'SPY', 'date']]
temp = qc[qc['date']>= '2021-03-29']
temp


,QCOM,SPY,date
0,120.2000,394.5900,2022-11-21
1,123.8500,396.0300,2022-11-18
2,121.4300,398.5100,2022-11-11
3,106.6900,376.3500,2022-11-04
4,119.2100,389.0200,2022-10-28
...,...,...,...
82,134.8904,408.3881,2021-04-30
83,131.6153,407.8400,2021-04-23
84,134.3170,408.3489,2021-04-16
85,136.6106,402.7022,2021-04-09


In [519]:
temp = temp.sort_values(by="date")
temp.head(10)

,QCOM,SPY,date
86,133.9089,392.0545,2021-04-01
85,136.6106,402.7022,2021-04-09
84,134.3170,408.3489,2021-04-16
83,131.6153,407.8400,2021-04-23
82,134.8904,408.3881,2021-04-30
81,133.9672,413.1052,2021-05-07
80,126.4841,407.6835,2021-05-14
79,127.7572,406.0785,2021-05-21
78,130.7504,411.0696,2021-05-28
77,131.2195,413.5749,2021-06-04


In [520]:


def createGrowthColumn(ticker, frame):

    pricesList = np.array(frame[ticker])
    pctChanges = np.diff(pricesList)/pricesList[:-1]

    currentInvestment , initialInvestment= 100, 100
    investmentValues = []

    for c in pctChanges[1:]: 
        currentInvestment = currentInvestment*(1+c) 
        investmentValues.append(currentInvestment)

    pctChanges = [np.nan ] + list(pctChanges)
    investmentValues = [ np.nan, initialInvestment ] + investmentValues

    invName= ticker+'_growth'
    pctName = ticker+'_pchange'
    frame[invName] = investmentValues
    frame[pctName] = pctChanges



createGrowthColumn('QCOM', temp)
createGrowthColumn('SPY', temp)
temp.head()


,QCOM,SPY,date,QCOM_growth,QCOM_pchange,SPY_growth,SPY_pchange
86,133.9089,392.0545,2021-04-01,NaN,NaN,NaN,NaN
85,136.6106,402.7022,2021-04-09,100.000000,0.020176,100.000000,0.027159
84,134.3170,408.3489,2021-04-16,98.321067,-0.016789,101.402202,0.014022
83,131.6153,407.8400,2021-04-23,96.343402,-0.020114,101.275831,-0.001246
82,134.8904,408.3881,2021-04-30,98.740800,0.024884,101.411937,0.001344


In [521]:
import plotly.express as px 

fig = px.line(temp, x='date', y=['QCOM_growth', 'SPY_growth' ],
 title="Growth in Qualcomm v S&P"
 )


fig.show()

What a great example! looks like there was a sharp rise in price starting in November, and by mid January Qualcomm had grown much more than the S&P. My hypothesis is that the majority of stock purchases will exhibit a similar pattern

In [522]:
import string 
print(string.ascii_uppercase)

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [523]:
def findSourceFrame(ticker):
    leadingLetter =ticker[0]

    relevantCols = ['date', ticker, 'SPY']

    if leadingLetter in 'ABCDEFG':
        rez = historicalPricesAG[relevantCols]
    elif leadingLetter in 'HIJKLMNOPQR':
        rez = historicalPricesHR[relevantCols]
    elif leadingLetter in 'STUVWXYZ':
        rez = historicalPricesSZ[relevantCols]
    else:
        return 'ticker leading letter is non-alpha'
    return rez




def generateHistoricalPricesTable(ticker, date):
    
    rez = findSourceFrame(ticker)

    rez = rez.sort_values(by="date")

    # start dataframe at purchase date 
    rez = rez[rez['date']>=date]
    
    # end dataframe at one year from purchase date 
    if len(rez)>52:
        rez = rez.head(52)

    return rez


def graphGrowthRates(frame, tickerFirst, tickerSecond='SPY'):
    yColumnList = [tickerFirst+"_growth", tickerSecond+"_growth"]
    fig = px.line(frame, x='date', y=yColumnList,
    title=f"Growth in {tickerFirst} v {tickerSecond}",
    height=500, width=800

    )
    fig.show()


# putting it all together 
def graphAlpha(ticker, date):

    histPrices =  generateHistoricalPricesTable(ticker, date)
    createGrowthColumn(ticker, histPrices)
    createGrowthColumn('SPY', histPrices)

    graphGrowthRates(histPrices, ticker)
 
    


In [524]:
graphAlpha("IBM", "2018-01-05")

Our all-in-one function for graphing alpha appears to be "functioning" properly. There is no obvious alpha here, but this is just a random stock and date. Instead of picking randomly, let's graph find some actual stock purachases and thier associated dates

In [525]:

def purchasesBySenator(senId):
    isSenator = transactions['senatorId']==senId
    isPurchase = transactions['type']=='Purchase'
    isStock = transactions['asset_type']=='Stock'
    rez = transactions[isSenator & isPurchase & isStock]

    return rez.loc[:, :'type']


tbPurchases = purchasesBySenator(100)
tbPurchases.tail(10)


,transaction_date,owner,ticker,asset_description,asset_type,type
1034,2021-03-29,Joint,ATHM,Autohome Inc.,Stock,Purchase
1035,2021-03-29,Joint,AME,"AMETEK, Inc.",Stock,Purchase
1036,2021-03-29,Joint,GOOGL,Alphabet Inc.,Stock,Purchase
1039,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
1040,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
1043,2021-03-12,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1056,2021-04-19,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1059,2021-04-16,Joint,ECOM,ChannelAdvisor Corporation,Stock,Purchase
1069,2021-04-07,Joint,OXY,Occidental Petroleum Corporation,Stock,Purchase
1082,2021-05-06,Joint,X,United States Steel Corporation,Stock,Purchase


In [526]:
graphAlpha("OXY", '2021-06-25')

In [527]:
alDate = reformatDate("03/16/2021")
graphAlpha('AA', alDate)


Alright so Tommy Tuberville had a great year in 2021, but the S&P was pretty lousy that year. Let's try a different Senator and another time period

In [528]:
senSample = senators.sample(n=30, random_state=69)['senatorId'].to_list()
# lets find the senator with the most purchases within a random sample of 30 senators 
maxPurchases= 0
maxSenator=0
for sen in senSample:
    currentPurchases = len(purchasesBySenator(sen))
    if currentPurchases >= maxPurchases:
        maxPurchases = currentPurchases
        maxSenator = sen

print(senators[senators['senatorId']==maxSenator])
purchasesBySenator(maxSenator).tail(10)

    row              senator  senatorId       party
34   35  David A Perdue , Jr        136  Republican


,transaction_date,owner,ticker,asset_description,asset_type,type
8382,2015-04-17,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8383,2015-04-23,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8384,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8385,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8387,2015-04-15,Joint,MOS,The Mosaic Company (NYSE),Stock,Purchase
8393,2015-05-12,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8394,2015-05-13,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8395,2015-05-14,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8397,2015-05-15,Joint,AXLL,Axiall Corporation (NYSE),Stock,Purchase
8398,2015-05-29,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase


In [529]:
graphAlpha("MOS", "2015-04-15")
print("this is is kind of a terrible trade")

this is is kind of a terrible trade


In [530]:
graphAlpha("KN", "2015-04-24")
print("this one has a bit of a pop, but also not a great trade")

this one has a bit of a pop, but also not a great trade


In [531]:
purchasesBySenator(136).iloc[-60:-50, :]

,transaction_date,owner,ticker,asset_description,asset_type,type
8284,2015-04-02,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8285,2015-04-02,Joint,AVP,Avon Products Inc. (NYSE),Stock,Purchase
8287,2015-04-02,Joint,JNJ,Johnson & Johnson (NYSE),Stock,Purchase
8288,2015-04-02,Joint,KO,The Coca-Cola Company (NYSE),Stock,Purchase
8290,2015-05-06,Joint,SYY,Sysco Corporation (NYSE),Stock,Purchase
8292,2015-05-11,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8294,2015-05-12,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8295,2015-01-07,Joint,AM,Antero Midstream Partners LP (NYSE),Stock,Purchase
8296,2015-01-14,Joint,AR,Antero Resources Corporation (NYSE),Stock,Purchase
8297,2015-01-15,Joint,BPL,"Buckeye Partners, L.P. (NYSE)",Stock,Purchase


In [532]:
graphAlpha("AM", "2015-01-15")

In [533]:
# purchasesBySenator(136).iloc[-90:-80, :]
purchasesBySenator(136).iloc[20:30]

,transaction_date,owner,ticker,asset_description,asset_type,type
2404,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2406,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2408,2020-03-09,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2409,2020-03-10,Joint,MSM,"MSC Industrial Direct Co., Inc.",Stock,Purchase
2410,2020-03-10,Joint,M,"Macy's, Inc.",Stock,Purchase
2411,2020-03-10,Joint,NWSA,News Corporation,Stock,Purchase
2412,2020-03-10,Joint,FOXA,Fox Corporation,Stock,Purchase
2413,2020-03-10,Joint,EAF,GrafTech International Ltd.,Stock,Purchase
2415,2020-03-13,Joint,ENB,Enbridge Inc.,Stock,Purchase
2418,2020-03-13,Joint,TRP,TC Energy Corporation,Stock,Purchase


In [534]:
graphAlpha("M", "2020-03-10")

here's a great example of why we need more information. Its true Macy's had a big run, but this run was 6 months after the purchase, and it was preceeded by a smaller run. Hard to say if the first run was the reason for the purcahse or not. Lets modify the graph alpha function to indicate all transacions (purcahses and sales) of one asset by a senator within the measurement window (1 year)

In [535]:
from dateutil.relativedelta import  relativedelta

transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])


def findRelevantTransactions(senId, date, ticker):
    startDate = pd.to_datetime(date)
    endDate = startDate + relativedelta(years=1)
    
    afterStartDate = transactions['transaction_date']>= startDate
    beforeEndDate = transactions['transaction_date']< endDate
    isStock = transactions['asset_type']=="Stock"
    isSenator = transactions['senatorId']==senId
    isTicker = transactions['ticker']==ticker

    rez = transactions[afterStartDate & beforeEndDate & isStock & isSenator & isTicker]

    return rez 



print("looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run ")

macysTransactions = findRelevantTransactions(136, "2020-03-10", "M")[['transaction_date', 'type','amount', 'senator', 'ticker']]
macysTransactions

looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run 


,transaction_date,type,amount,senator,ticker
1980,2020-04-14,Sale (Full),"$15,001 - $50,000","David A Perdue , Jr",M
2410,2020-03-10,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M
2466,2020-03-20,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M


In [556]:
# from datetime import datetime
import datetime
#generate map of dates and transaction types 
dates = macysTransactions['transaction_date'].to_list()
transTypes = macysTransactions['type'].to_list()

#calculate the closest friday to merge with historical prices table (weekly data)
def nearestFriday(inputDate):
    fridayDelta = datetime.timedelta( (4-inputDate.weekday()) % 7 )
    nearestFriday = inputDate + fridayDelta
    nearestFridayString = datetime.datetime.strftime(nearestFriday , "%Y-%m-%d")
    return nearestFridayString

fridayDates  = [nearestFriday(d) for d in dates]

transactionMap = dict(zip(fridayDates, transTypes))



transactionMap 

{'2020-04-17': 'Sale (Full)',
 '2020-03-13': 'Purchase',
 '2020-03-20': 'Purchase'}

In [587]:

# putting it all together
# find relevant transactions
def historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput ):
    relevantTransactions = findRelevantTransactions(senatorIdInput, dateInput,tickerInput)

    dates = relevantTransactions['transaction_date'].to_list()
    transTypes = relevantTransactions['type'].to_list()
    fridayDates  = [nearestFriday(d) for d in dates]
    transactionMap = dict(zip(fridayDates, transTypes)) 

    hist = generateHistoricalPricesTable(tickerInput, dateInput)

    hist['date']  = hist['date'].dt.strftime("%Y-%m-%d")

    mappedColumn = []
    isTransaction = []



    for date in hist['date'].to_list():
        if date in transactionMap.keys():
            mappedColumn.append(transactionMap[date])
        else:
            mappedColumn.append(np.nan)
            # mappedColumn.append('fill')

    hist['transactionType'] = mappedColumn
    hist['transactionValue'] = np.where(hist['transactionType'].notnull(), hist['M'], np.nan)


    return hist


temp = historicalPricesWithTransaction(136,  "2020-03-10", "M" )
temp.head()


,date,M,SPY,transactionType,transactionValue
141,2020-03-13,7.7445,257.7998,Purchase,7.7445
140,2020-03-20,5.8060,220.3585,Purchase,5.8060
139,2020-03-27,5.3334,244.0702,NaN,NaN
138,2020-04-03,4.6390,239.0331,NaN,NaN
137,2020-04-09,6.4232,267.9359,NaN,NaN


In [602]:
import plotly.graph_objects as go 


# px.scatter(temp, x='date', y=['M','transactionValue'],color='transactionType')
fig = px.line(temp, x='date', y=['M'])
# fig.add_trace(go.Scatter(temp, x='date', y=['M','transactionValue'],color='transactionType'))
# fig.add_trace(go.Scatter( x=temp['date'], y=temp['transactionValue'],fill=temp['transactionType']))
fig.add_trace(go.Scatter( x=temp['date'], y=temp['transactionValue'], mode="markers",))


fig.show()

In [ ]:
f = px.data.iris()

fig = px.line(temp, x='date', y=['M'])


fig.add_trace(
    #    go.Scatter(temp, x='date', y=['M','transactionValue'],color='transactionType')
    #    go.Scatter( x=temp['date'], y=temp['transactionValue'],color=temp['transactionType'])
       go.Scatter( x=temp['date'], y=temp['transactionValue'])
)
fig.show()